1. Создаем таблицы по условию

2. Загружаем данные в таблицу customers

<img src="cust.png" width="600">

3. Загружаем данные в таблицу orders

<img src="order.png" width="600">

4. Загружаем данные в таблицу products

<img src="prod.png" width="600">

5. Загружаем данные в таблицу order_items

<img src="ord_it.png" width="600">

Далее выполняем запросы. 

<img src="1_.png" width="600">

<img src="2_.png" width="600">

<img src="3_.png" width="600">

<img src="4_.png" width="600">

<img src="5_.png" width="600">

<img src="6_.png" width="600">

<img src="7_.png" width="600">

<img src="8_.png" width="600">